In [2]:
import os
import librosa
import numpy as np
import sqlite3

In [2]:
def get_size_kb(file_path):
    return os.path.getsize(file_path)/1024

def calculate_silence_ratio(file_path, silence_threshold=1e-4, frame_length=2048, hop_length=512):
    # Load the audio file
    y, sr = librosa.load(file_path, sr=None)

    # Calculate the total duration of the audio in seconds
    total_duration = len(y) / sr

    # Calculate the short-term energy for each frame
    energy = np.array([
        np.sum(np.abs(y[i:i+frame_length]**2))
        for i in range(0, len(y), hop_length)
    ])

    # Define silence as frames where energy is below the threshold
    silent_frames = np.sum(energy < silence_threshold)

    # Calculate the duration of silent frames in seconds
    silent_duration = (silent_frames * hop_length) / sr

    # Calculate the silence ratio
    silence_ratio = silent_duration / total_duration

    return silence_ratio

def get_wav_informations(file_path):
    # name = os.path.basename(file_path)
    size = get_size_kb(file_path)
    wav, sr = librosa.load(file_path)
    duration = librosa.get_duration(y=wav, sr=sr)
    channel = len(wav.shape)
    silence_ratio = calculate_silence_ratio(file_path)
    size = get_size_kb(file_path)
    # creation_time = os.path.getctime(file_path)
    return size, sr, channel, duration, silence_ratio, wav

In [3]:
folder_path = "/run/user/1000/gvfs/smb-share:server=10.70.115.81,share=ghiam/2024/01/01"

In [7]:
con = sqlite3.connect("database.db")

In [8]:
cur = con.cursor()

In [9]:
cur.execute("CREATE TABLE wav(file_name, file_path, sr, channel, data)")
cur.execute("CREATE TABLE wav_feature(file_name, file_size, duration, silence_ratio)")
cur.execute("CREATE TABLE wav_nlp(file_name, sumaration, content, tag)")

In [10]:
con.close()

In [9]:
if os.path.exists(folder_path):
    # Loop through each file in the folder
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        # remove noise file before processing
        if get_size_kb(file_path) > 1:        
        # Only process files, not subdirectories
            if os.path.isfile(file_path):
                file_size, sr, channel, duration, silence_ratio, data = get_wav_informations(file_path)
                print(f"{file_name}: {file_size}, {sr}, {channel}, {duration}, {silence_ratio}, {data}")
                cur.execute(f"""INSERT INTO wav VALUES ("{file_name}", "{file_path}", {sr}, {channel}, "{data}")""")
                # cur.execute(f"""INSERT INTO wav_feature VALUES ("{file_name}", "{file_size}", {duration}, {silence_ratio})""")

                con.commit()


external-7411-+84325556721-20240101-095230-1704077550.450390.wav: 720.98046875, 22050, 1, 46.14, 0.0, [-0.00182781 -0.0021723  -0.00176293 ... -0.01008465 -0.00699104
 -0.00325698]
external-7411-+84922364552-20240101-111228-1704082348.450616.wav: 8050.35546875, 22050, 1, 515.22, 0.0, [-9.0146423e-06 -2.3860095e-05  6.6107976e-05 ... -1.9252056e-03
 -1.6091906e-03 -7.9235178e-04]
in-+842836223636-+84325556721-20240101-095212-1704077532.450384.wav: 720.98046875, 22050, 1, 46.14, 0.0, [-0.001627   -0.00205733 -0.00185753 ... -0.01008465 -0.00699104
 -0.00325698]
in-+842836223636-+84922364552-20240101-111206-1704082326.450577.wav: 8044.10546875, 22050, 1, 514.82, 0.00012431529466609687, [-0.00072652 -0.0011639  -0.00121207 ... -0.00192521 -0.00160919
 -0.00079235]
out-90326406399-7445-20240101-145048-1704095448.451155.wav: 572.23046875, 22050, 1, 36.62, 0.0, [0.00073008 0.00117282 0.0013009  ... 0.00389087 0.00385798 0.00224157]
out-90326745134-7486-20240101-132058-1704090058.450803.wav: 8